In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [71]:
df_reviews_raw = pd.read_csv("GameReviewsFinal.csv")
df_stats_raw = pd.read_csv("GameStatsFinal.csv")

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,5,6,7,8,10,11,12,13,14,15,16,17,21,22,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [72]:
df_reviews_raw.shape
df_reviews_raw.head(10)


# First two columns look like indexes so I'll be removing them after
# extracting the appid
# Also, dont think we need recommendation id
# author may be useful but not sure currently so removing it
# leaving the other columns

,Unnamed: 0,Unnamed: 0.1,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,timestamp_dev_responded,developer_response,appid
0,0.0,1155810_1,56542917,76561197965909487,english,"Unfortunately, at this price point I was expec...",1572942648,1572942648,False,11,2,0.658271,0,True,False,True,1.60828e+09,This! Completely fair review! Santa's Visit la...,1155810
1,1.0,1155810_2,83106481,76561197983047952,english,A cute sandbox experience. Fun for the Holiday...,1608896024,1608896024,True,7,0,0.599486,0,True,False,True,NaN,NaN,1155810
2,2.0,1155810_3,80462795,76561198003402530,english,While its a very short experience it has fanta...,1606426930,1606426930,True,7,0,0.571652,0,True,False,True,NaN,NaN,1155810
3,3.0,1155810_4,60567594,76561198094159267,english,I got Santa trapped in Purgatory.\n10/10 holid...,1577329940,1577329940,True,7,1,0.571652,0,True,False,True,NaN,NaN,1155810
4,4.0,1155810_5,106470307,76561198066590240,english,[h1]Experienced on the Oculus Quest 2[/h1]\n\n...,1640408475,1640408475,True,3,1,0.566025,0,True,False,True,NaN,NaN,1155810
5,5.0,1155810_6,57259115,76561198062396161,english,"Multiple crashes, doesn't like WMR, had to ref...",1573877503,1573877503,False,3,1,0.542957,0,True,False,True,1.57442e+09,"Hello! At the time of this writing, as indica...",1155810
6,6.0,1155810_7,56452383,76561197970431718,english,I'm a kid again playing Santa Have the Christ...,1572832099,1572832099,True,9,3,0.520549,0,True,False,True,NaN,NaN,1155810
7,7.0,1145740_1,56441818,76561198052645607,english,Game is a beat the clock game. Very little ani...,1572819777,1572819777,False,8,0,0.548528,0,True,False,True,NaN,NaN,1145740
8,8.0,1145740_2,76832294,76561198018222615,english,Game is broken and no fixes since 2019. DONT BUY,1601501799,1601501799,False,3,0,0.545455,0,True,False,True,NaN,NaN,1145740
9,9.0,1145740_3,84911987,76561198075822106,english,Developer quit developing this after 1 patch. ...,1610978667,1610978667,False,3,0,0.542039,0,True,False,True,NaN,NaN,1145740


In [73]:
df_reviews_raw[['appid', 'rev_id']] = df_reviews_raw['Unnamed: 0.1'].str.split("_", expand=True)
df_reviews_raw.head()

,Unnamed: 0,Unnamed: 0.1,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,timestamp_dev_responded,developer_response,appid,rev_id
0,0.0,1155810_1,56542917,76561197965909487,english,"Unfortunately, at this price point I was expec...",1572942648,1572942648,False,11,2,0.658271,0,True,False,True,1.60828e+09,This! Completely fair review! Santa's Visit la...,1155810,1
1,1.0,1155810_2,83106481,76561197983047952,english,A cute sandbox experience. Fun for the Holiday...,1608896024,1608896024,True,7,0,0.599486,0,True,False,True,NaN,NaN,1155810,2
2,2.0,1155810_3,80462795,76561198003402530,english,While its a very short experience it has fanta...,1606426930,1606426930,True,7,0,0.571652,0,True,False,True,NaN,NaN,1155810,3
3,3.0,1155810_4,60567594,76561198094159267,english,I got Santa trapped in Purgatory.\n10/10 holid...,1577329940,1577329940,True,7,1,0.571652,0,True,False,True,NaN,NaN,1155810,4
4,4.0,1155810_5,106470307,76561198066590240,english,[h1]Experienced on the Oculus Quest 2[/h1]\n\n...,1640408475,1640408475,True,3,1,0.566025,0,True,False,True,NaN,NaN,1155810,5


In [74]:
df_reviews_clean = df_reviews_raw.drop(['Unnamed: 0', 'Unnamed: 0.1', 'recommendationid', 'author', 'rev_id'], axis=1)
df_reviews_clean.head()

,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,timestamp_dev_responded,developer_response,appid
0,english,"Unfortunately, at this price point I was expec...",1572942648,1572942648,False,11,2,0.658271,0,True,False,True,1.60828e+09,This! Completely fair review! Santa's Visit la...,1155810
1,english,A cute sandbox experience. Fun for the Holiday...,1608896024,1608896024,True,7,0,0.599486,0,True,False,True,NaN,NaN,1155810
2,english,While its a very short experience it has fanta...,1606426930,1606426930,True,7,0,0.571652,0,True,False,True,NaN,NaN,1155810
3,english,I got Santa trapped in Purgatory.\n10/10 holid...,1577329940,1577329940,True,7,1,0.571652,0,True,False,True,NaN,NaN,1155810
4,english,[h1]Experienced on the Oculus Quest 2[/h1]\n\n...,1640408475,1640408475,True,3,1,0.566025,0,True,False,True,NaN,NaN,1155810


In [75]:
df_reviews_clean.isna().sum()

language                            0
review                            140
timestamp_created                   0
timestamp_updated                   0
voted_up                            0
votes_up                            0
votes_funny                         0
weighted_vote_score                 0
comment_count                       0
steam_purchase                      0
received_for_free                   0
written_during_early_access         0
timestamp_dev_responded         39351
developer_response             292175
appid                               0
dtype: int64

In [76]:
# check for missing reviews and removing them

subset = ['review', 'appid']
df_reviews_clean.dropna(subset=subset, axis=0, inplace=True)

In [77]:
# Converting the review column to string
df_reviews_clean['review'] = df_reviews_clean['review'].astype('str')
df_reviews_clean['language'] = df_reviews_clean['language'].astype('str')

In [78]:
df_reviews_clean.info()
df_reviews_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 302453 entries, 0 to 302592
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   language                     302453 non-null  object
 1   review                       302453 non-null  object
 2   timestamp_created            302453 non-null  object
 3   timestamp_updated            302453 non-null  object
 4   voted_up                     302453 non-null  object
 5   votes_up                     302453 non-null  object
 6   votes_funny                  302453 non-null  object
 7   weighted_vote_score          302453 non-null  object
 8   comment_count                302453 non-null  object
 9   steam_purchase               302453 non-null  object
 10  received_for_free            302453 non-null  object
 11  written_during_early_access  302453 non-null  object
 12  timestamp_dev_responded      263125 non-null  object
 13  developer_resp

,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,timestamp_dev_responded,developer_response,appid
0,english,"Unfortunately, at this price point I was expec...",1572942648,1572942648,False,11,2,0.658271,0,True,False,True,1.60828e+09,This! Completely fair review! Santa's Visit la...,1155810
1,english,A cute sandbox experience. Fun for the Holiday...,1608896024,1608896024,True,7,0,0.599486,0,True,False,True,NaN,NaN,1155810
2,english,While its a very short experience it has fanta...,1606426930,1606426930,True,7,0,0.571652,0,True,False,True,NaN,NaN,1155810
3,english,I got Santa trapped in Purgatory.\n10/10 holid...,1577329940,1577329940,True,7,1,0.571652,0,True,False,True,NaN,NaN,1155810
4,english,[h1]Experienced on the Oculus Quest 2[/h1]\n\n...,1640408475,1640408475,True,3,1,0.566025,0,True,False,True,NaN,NaN,1155810


In [79]:
df_stats_raw.head(10)

,Unnamed: 0,Unnamed: 0.1,appid,name,developer,publisher,score_rank,positive,negative,userscore,...,ccu,languages,genre,tags,num_reviews,review_score,review_score_desc,total_positive,total_negative,total_reviews
0,0,0,250340,Blockland,"Eric Hartman, Ben Garney, Anthony Rotondo",Step 1 Games LLC,NaN,4042,1499,0,...,46,English,"Action, Indie, Simulation","Sandbox, Building, Multiplayer, LEGO, Open Wor...",4,6,Mostly Positive,1678,676,2354
1,1,1,429570,The Walking Dead: Michonne - A Telltale Minise...,Telltale Games,Skybound Games,NaN,4754,1292,0,...,9,"English, French, German, Spanish - Spain, Port...",Adventure,"Adventure, Zombies, Story Rich, Choices Matter...",1,6,Mostly Positive,1407,467,1874
2,2,2,775500,SCARLET NEXUS,BANDAI NAMCO Studios Inc.,BANDAI NAMCO Entertainment,NaN,8666,1523,0,...,367,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, RPG","Action, Anime, RPG, Adventure, Action RPG, JRP...",20,8,Very Positive,2892,346,3238
3,3,3,272470,The Incredible Adventures of Van Helsing II,NeocoreGames,NeocoreGames,NaN,1581,725,0,...,7,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Indie, RPG","RPG, Action, Adventure, Hack and Slash, Action...",20,5,Mixed,479,237,716
4,4,4,222750,Wargame: Airland Battle,Eugen Systems,Eugen Systems,NaN,3670,505,0,...,47,"English, German, French, Italian, Spanish - Sp...","Indie, Simulation, Strategy","Strategy, RTS, Military, Cold War, Wargame, Re...",1,8,Very Positive,1921,276,2197
5,5,5,1285670,Post Void,YCJY Games,YCJY Games,NaN,6474,145,0,...,23,English,"Action, Indie","FPS, Psychedelic, Action Roguelike, Fast-Paced...",20,9,Overwhelmingly Positive,4648,84,4732
6,6,6,547680,Guild Quest,Hyper Hippo Games,Hyper Hippo Games,NaN,1151,556,0,...,13,English,"Casual, Free to Play, Indie, RPG, Simulation, ...","Free to Play, Strategy, Casual, Simulation, Ea...",20,6,Mostly Positive,159,53,212
7,7,7,438030,Dead6hot,Chuck Productions,New Reality Games,NaN,212,259,0,...,39,English,"Action, Casual","Action, Casual, 2D, Pixel Graphics, Indie",20,4,Mostly Negative,35,70,105
8,8,8,1238060,Dead Space 3,Visceral Games,Electronic Arts,NaN,5889,1774,0,...,299,"English, French, Italian, German, Spanish - Sp...",Action,"Action, Horror, Online Co-Op, Sci-fi, Shooter,...",12,6,Mostly Positive,1722,604,2326
9,9,9,1140620,CosmicBreak Universal,"CyberStep, Inc.","CyberStep, Inc.",NaN,644,453,0,...,152,"English, Japanese, Traditional Chinese","Action, Free to Play, Massively Multiplayer, RPG","Free to Play, Action, Massively Multiplayer, R...",0,0,No user reviews,0,0,0


In [80]:
# Merging reviews dataframe and stats dataframe on appid

df_reviews_clean = df_reviews_clean.merge(df_stats_raw, on='appid')
print(df_reviews_clean.head())
df_reviews_clean.info()

  language                                             review  \
0  english  [h1]Buy and play this BEFORE Soko Loco Deluxe....   
1  english  [h1]Buy and play this BEFORE Soko Loco Deluxe....   
2  english  I ran into a bug where a house construction ne...   
3  english  I ran into a bug where a house construction ne...   
4  english  A sweet little tycoon / transport game!\nSuper...   

  timestamp_created timestamp_updated voted_up votes_up votes_funny  \
0        1580149960        1580149975     True        4           0   
1        1580149960        1580149975     True        4           0   
2        1609053807        1609053807    False        1           1   
3        1609053807        1609053807    False        1           1   
4        1652991517        1652991517     True        0           0   

  weighted_vote_score comment_count steam_purchase  ... ccu languages  \
0            0.549797             0           True  ...   0   English   
1            0.549797             0 

In [81]:
# Let us keep only the columns that'd be useful for us

my_reviews = df_reviews_clean[['appid', 'genre', 'tags', 'review']]
my_reviews.head()

,appid,genre,tags,review
0,1135200,"Indie, Simulation, Strategy","Strategy, Indie, Simulation, Trains, Transport...",[h1]Buy and play this BEFORE Soko Loco Deluxe....
1,1135200,"Indie, Simulation, Strategy","Strategy, Indie, Simulation, Trains, Transport...",[h1]Buy and play this BEFORE Soko Loco Deluxe....
2,1135200,"Indie, Simulation, Strategy","Strategy, Indie, Simulation, Trains, Transport...",I ran into a bug where a house construction ne...
3,1135200,"Indie, Simulation, Strategy","Strategy, Indie, Simulation, Trains, Transport...",I ran into a bug where a house construction ne...
4,1135200,"Indie, Simulation, Strategy","Strategy, Indie, Simulation, Trains, Transport...",A sweet little tycoon / transport game!\nSuper...


In [82]:
# Seems like there are duplicate values

my_reviews = my_reviews.drop_duplicates(ignore_index=True)
print(my_reviews.head())
my_reviews.info()

     appid                        genre  \
0  1135200  Indie, Simulation, Strategy   
1  1135200  Indie, Simulation, Strategy   
2  1135200  Indie, Simulation, Strategy   
3  1231120      Casual, Indie, Strategy   
4  1231120      Casual, Indie, Strategy   

                                                tags  \
0  Strategy, Indie, Simulation, Trains, Transport...   
1  Strategy, Indie, Simulation, Trains, Transport...   
2  Strategy, Indie, Simulation, Trains, Transport...   
3  Casual, Strategy, Indie, Puzzle, Puzzle-Platfo...   
4  Casual, Strategy, Indie, Puzzle, Puzzle-Platfo...   

                                              review  
0  [h1]Buy and play this BEFORE Soko Loco Deluxe....  
1  I ran into a bug where a house construction ne...  
2  A sweet little tycoon / transport game!\nSuper...  
3  Lovely work! My 6 year old daughter saw me pla...  
4  If you enjoy 2D puzzle platformers, this game ...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246652 entries, 0 to 246

In [83]:
my_reviews['genre'] = my_reviews['genre'].str.split(', ')
my_reviews.head()

,appid,genre,tags,review
0,1135200,"[Indie, Simulation, Strategy]","Strategy, Indie, Simulation, Trains, Transport...",[h1]Buy and play this BEFORE Soko Loco Deluxe....
1,1135200,"[Indie, Simulation, Strategy]","Strategy, Indie, Simulation, Trains, Transport...",I ran into a bug where a house construction ne...
2,1135200,"[Indie, Simulation, Strategy]","Strategy, Indie, Simulation, Trains, Transport...",A sweet little tycoon / transport game!\nSuper...
3,1231120,"[Casual, Indie, Strategy]","Casual, Strategy, Indie, Puzzle, Puzzle-Platfo...",Lovely work! My 6 year old daughter saw me pla...
4,1231120,"[Casual, Indie, Strategy]","Casual, Strategy, Indie, Puzzle, Puzzle-Platfo...","If you enjoy 2D puzzle platformers, this game ..."


In [86]:
genre_expanded = my_reviews['genre'].explode().groupby(level=0).value_counts()\
.unstack(fill_value=0).reindex(my_reviews.index, fill_value=0)
genre_expanded
genre_expanded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246652 entries, 0 to 246651
Data columns (total 43 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   Accounting             246652 non-null  int64
 1   Action                 246652 non-null  int64
 2   Adventure              246652 non-null  int64
 3   Animation & Modeling   246652 non-null  int64
 4   Atmospheric            246652 non-null  int64
 5   Audio Production       246652 non-null  int64
 6   Blood                  246652 non-null  int64
 7   Casual                 246652 non-null  int64
 8   Classic                246652 non-null  int64
 9   Dark                   246652 non-null  int64
 10  Demons                 246652 non-null  int64
 11  Design & Illustration  246652 non-null  int64
 12  Difficult              246652 non-null  int64
 13  Early Access           246652 non-null  int64
 14  Education              246652 non-null  int64
 15  Free to Play     

In [87]:
df_concat = pd.concat([my_reviews, genre_expanded], axis=1)

In [88]:
df_concat.head()

,appid,genre,tags,review,Accounting,Action,Adventure,Animation & Modeling,Atmospheric,Audio Production,...,Software Training,Sports,Strategy,Survival,Survival Horror,Utilities,Video Production,Violent,Web Publishing,Zombies
0,1135200,"[Indie, Simulation, Strategy]","Strategy, Indie, Simulation, Trains, Transport...",[h1]Buy and play this BEFORE Soko Loco Deluxe....,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1135200,"[Indie, Simulation, Strategy]","Strategy, Indie, Simulation, Trains, Transport...",I ran into a bug where a house construction ne...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1135200,"[Indie, Simulation, Strategy]","Strategy, Indie, Simulation, Trains, Transport...",A sweet little tycoon / transport game!\nSuper...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1231120,"[Casual, Indie, Strategy]","Casual, Strategy, Indie, Puzzle, Puzzle-Platfo...",Lovely work! My 6 year old daughter saw me pla...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1231120,"[Casual, Indie, Strategy]","Casual, Strategy, Indie, Puzzle, Puzzle-Platfo...","If you enjoy 2D puzzle platformers, this game ...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
## Some EDA on the reviews

review_length = my_reviews['review'].str.len()
print(review_length.mean())
print(review_length.std())
print(review_length.max())
print(review_length.min())

In [ ]:
# Strings with length 1 don't make any sense. Lets check some strings with length less than 10

# First I'll add review_length as a column
my_reviews['review_length'] = review_length

# Now lets check the reviews
for i in range(10):
    print(my_reviews.loc[my_reviews['review_length'] == i].head()['review'])


In [ ]:
# I think that even comments like "no" and "yes" can be used to gauge sentiments.
# So I'll only remove comments that are just a single character

my_reviews.drop(my_reviews[my_reviews['review_length'] < 2].index, inplace = True)

In [ ]:
print(my_reviews.shape)
print(my_reviews['review_length'].mean())
print(my_reviews['review_length'].std())
print(my_reviews['review_length'].max())
print(my_reviews['review_length'].min())

In [ ]:
my_pattern = r'\b[^\d\W][^\d\W]+\b'
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000, token_pattern=my_pattern, stop_words=ENGLISH_STOP_WORDS)\
.fit(my_reviews['review'])

In [ ]:
X_txt = vectorizer.transform(my_reviews['review'])

In [ ]:
X = pd.DataFrame(X_txt.toarray(), columns=vectorizer.get_feature_names())
print('Top 5 rows of the DataFrame: ', X.head())

In [ ]:
####################
####################

count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern=my_pattern,
                                   lowercase=True,
                                   max_df = 0.6, max_features=5000)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern=my_pattern,
                                   lowercase=True,
                                   max_df = 0.6, max_features=5000)

cv_data = count_vectorizer.fit_transform(my_reviews['review'])
tfidf_data = tfidf_vectorizer.fit_transform(my_reviews['review'])

In [ ]:
####################
####################

for_wordcloud = count_vectorizer.get_feature_names()
for_wordcloud = for_wordcloud
for_wordcloud_str = ' '.join(for_wordcloud)

wordcloud = WordCloud(width=800, height=400, background_color ='white',
                      min_font_size = 7).generate(for_wordcloud_str)

plt.figure(figsize=(20, 10), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)
 
plt.show()

In [ ]:
Sum_of_squared_distances = []
K = range(2,10)

for k in K:
    km = KMeans(n_clusters=k, max_iter=200, n_init=10)
    km = km.fit(X_txt)
    Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Optimal k')
plt.show()

In [ ]:
true_k = 6
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=200, n_init=10)
model.fit(X_txt)
labels=model.labels_
